In [191]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 294 kB 252 kB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 356 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 719 kB/s eta 0:00:01
     |████████████████████████████████| 249 kB 475 kB/s eta 0:00:01
     |████████████████████████████████| 106 kB 470 kB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 455 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [216]:
import pandas as pd
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from dateutil.relativedelta import relativedelta
import re
import seaborn as sns

In [217]:
# Загрузка данных
data = pd.read_csv('ds_dirty_fin_202410041147.csv', low_memory=False)

In [218]:
data = data[[c for c
        in list(data)
        if len(data[c].unique()) > 1]]
data.drop_duplicates(inplace=True)

In [219]:
data

,client_id,client_first_name,client_middle_name,client_last_name,client_fio_full,client_bday,client_bplace,client_cityzen,client_resident_cd,client_gender,...,fin_loan_value,fin_loan_debt,fin_loan_percent,fin_loan_begin_dt,fin_loan_end_dt,stream_favorite_show,stream_duration,create_date,update_date,source_cd
0,596118586,Меджид,Кузарович,Евдунов,Евдунов Меджид Кузарович,1992-08-23,ДЕР. КИРГИНЦЕВО АРОМАШЕВСКОГО Р-НА ТЮМЕНСКОЙ ОБЛ.,RUS,Д,М,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-16 05:10:57.904,2020-07-16 05:10:57.904,Bank
1,747208737,Мансыр,Хирсанович,Повод,Повод Мансыр Хирсанович,1987-03-15,NaN,NaN,,М,...,NaN,NaN,NaN,NaN,NaN,Star Wars: A New Hope,1187.0,2020-11-13 21:33:19.620,2020-11-13 21:33:19.620,Stream
2,625104053,НУРДИНБЕК,УВАЙСОВИЧ,БУРНАС,БУРНАС НУРДИНБЕК УВАЙСОВИЧ,1961-04-05,СЕЛО ДИОРОВА ТЯЧЕВСКОГО РАЙОНА ЗАКАРПАТСКОЙ ОБ...,RUS,,М,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-06 20:41:55.362,2022-09-06 20:41:55.362,Telco
3,624196811,Афаят,Гордиевна,Бондажевская,Бондажевская Афаят Гордиевна,1976-06-01,NaN,NaN,,Ж,...,NaN,NaN,NaN,NaN,NaN,Остаться в живых,35.0,2021-08-17 19:00:45.024,2021-08-17 19:00:45.024,Stream
4,177500487,Сахия,Фидаевна,Разуева,Разуева Сахия Фидаевна,1961-07-31,NaN,NaN,,Ж,...,NaN,NaN,NaN,NaN,NaN,Начало,834.0,2022-11-05 05:37:58.131,2022-12-04 00:00:09.068,Stream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005525,341669039,Мейрам,Авхадиевич,Маркеевич,Маркеевич Мейрам Авхадиевич,1939-07-14,NaN,NaN,,М,...,NaN,NaN,NaN,NaN,NaN,Крестный отец 2,926.0,2021-12-22 20:40:33.787,2021-12-22 20:40:33.787,Stream
1005526,790628473,Багжат,Санжасамсоновна,Мазова,Мазова Багжат Санжасамсоновна,1994-10-24,NaN,NaN,,Ж,...,NaN,NaN,NaN,NaN,NaN,Рик и Морти,1198.0,2023-09-08 08:40:55.500,2023-09-08 08:40:55.500,Stream
1005527,978076125,Динаида,Казимировна,Бубнева,Бубнева Динаида Казимировна,1995-09-03,NaN,NaN,,Ж,...,NaN,NaN,NaN,NaN,NaN,Матрица,39.0,2022-10-01 18:33:55.676,2022-10-01 18:33:55.676,Stream
1005528,224784654,Ненила,Уудовна,Жужук,Жужук Ненила Уудовна,1979-01-02,NaN,NaN,,Ж,...,NaN,NaN,NaN,NaN,NaN,Побег,214.0,2021-06-02 05:44:56.775,2021-11-05 15:04:17.259,Stream


In [220]:
data.nunique()

client_id               985864
client_first_name        50093
client_middle_name       93756
client_last_name        418546
client_fio_full         473820
client_bday              31807
client_bplace           222881
client_cityzen             202
client_resident_cd           3
client_gender                3
client_marital_cd            3
client_graduate              3
client_child_cnt             4
client_mil_cd                3
client_zagran_cd             3
client_inn              244733
client_snils            243155
client_vip_cd              172
contact_vc               50911
contact_tg               50935
contact_other                3
contact_email           258085
contact_phone           256731
addr_region                 96
addr_country                60
addr_zip                 19460
addr_street              34107
addr_house                4020
addr_body                  485
addr_flat                 2225
addr_area                 6203
addr_loc                   627
addr_cit

In [221]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 987673 entries, 0 to 1005529
Data columns (total 46 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   client_id             987673 non-null  int64  
 1   client_first_name     944018 non-null  object 
 2   client_middle_name    944039 non-null  object 
 3   client_last_name      944091 non-null  object 
 4   client_fio_full       980883 non-null  object 
 5   client_bday           987631 non-null  object 
 6   client_bplace         857055 non-null  object 
 7   client_cityzen        857055 non-null  object 
 8   client_resident_cd    987479 non-null  object 
 9   client_gender         987479 non-null  object 
 10  client_marital_cd     987479 non-null  object 
 11  client_graduate       519446 non-null  object 
 12  client_child_cnt      657037 non-null  float64
 13  client_mil_cd         987479 non-null  object 
 14  client_zagran_cd      987479 non-null  object 
 15  clie

In [222]:
REGEXP_SYMBOLS = r'[^a-zA-Zа-яА-Я\-\ё\Ё\ ]'

In [223]:
def source_priority(source: str):
    SOURCE_PRIORITY = {'bank': 6, 'stream': 0, 'telco': 4, 'bank2': 5, 'fitness': 1, 'isp': 3, 'travel': 2}
    return SOURCE_PRIORITY.get(source.lower() if isinstance(source, str) else source, -1)

def get_client_bd(date):
    if not pd.isnull(date):
        r = relativedelta(pd.to_datetime('today'), date).years
        if r > 100 or r < 0:
            return None
        return int(relativedelta(pd.to_datetime('today'), date).years)
    return None


In [224]:
def preprocess_data(df: pd.DataFrame):
    df['source_priority'] = df['source_cd'].apply(source_priority)
 # Преобразование столбца 'update_date' в формат datetime и удаление некорректных дат
    df['create_date'] = pd.to_datetime(df['update_date'], errors='coerce')
    df['update_date'] = pd.to_datetime(df['update_date'], errors='coerce')
    # df['client_bday'] = pd.DatetimeIndex(pd.to_datetime(df['client_bday'], errors='coerce')).year
    df['client_bday'] = pd.to_datetime(df['client_bday'], errors='coerce')
    df['client_yo'] = df['client_bday'].apply(get_client_bd)
    mean_age = int(df['client_yo'].mean())

    df['client_yo'] = df['client_yo'].fillna(mean_age)

    # Очистка и стандартизация строковых данных
    for col in df.select_dtypes(include=['object']).columns:
        if col != 'client_yo' and col != 'source_priority':
            df[col] = df[col].str.lower().fillna('').str.strip()  # Приведение к нижнему регистру, удаление пробелов

    # Очистка имени от чисел и мусорных символов
    df['client_first_name'] = df['client_first_name'].apply(lambda x: re.sub(REGEXP_SYMBOLS, '', x))
    # Очистка имени от чисел и мусорных символов
    df['client_middle_name'] = df['client_middle_name'].apply(lambda x: re.sub(REGEXP_SYMBOLS, '', x))
    # Очистка имени от чисел и мусорных символов
    df['client_last_name'] = df['client_last_name'].apply(lambda x: re.sub(REGEXP_SYMBOLS, '', x))
    # Очистка имени от чисел и мусорных символов
    df['client_fio_full'] = df['client_fio_full'].apply(lambda x: re.sub(REGEXP_SYMBOLS, '', x))

    return df

In [225]:
# Предбработка данных
data = preprocess_data(data)

In [226]:
data.nunique()

client_id               985864
client_first_name        26307
client_middle_name       48607
client_last_name        232558
client_fio_full         264749
client_bday              30105
client_bplace           220578
client_cityzen             203
client_resident_cd           3
client_gender                3
client_marital_cd            3
client_graduate              4
client_child_cnt             4
client_mil_cd                3
client_zagran_cd             3
client_inn              244733
client_snils            243155
client_vip_cd              173
contact_vc               50912
contact_tg               50936
contact_other                4
contact_email           258086
contact_phone           256731
addr_region                 96
addr_country                60
addr_zip                 19458
addr_street              30447
addr_house                3789
addr_body                  451
addr_flat                 2123
addr_area                 5258
addr_loc                   603
addr_cit

In [227]:
data['source_priority'].unique()

array([ 6,  0,  4,  5,  1,  3, -1,  2])

In [228]:
data.columns

Index(['client_id', 'client_first_name', 'client_middle_name',
       'client_last_name', 'client_fio_full', 'client_bday', 'client_bplace',
       'client_cityzen', 'client_resident_cd', 'client_gender',
       'client_marital_cd', 'client_graduate', 'client_child_cnt',
       'client_mil_cd', 'client_zagran_cd', 'client_inn', 'client_snils',
       'client_vip_cd', 'contact_vc', 'contact_tg', 'contact_other',
       'contact_email', 'contact_phone', 'addr_region', 'addr_country',
       'addr_zip', 'addr_street', 'addr_house', 'addr_body', 'addr_flat',
       'addr_area', 'addr_loc', 'addr_city', 'addr_str', 'fin_rating',
       'fin_loan_limit', 'fin_loan_value', 'fin_loan_debt', 'fin_loan_percent',
       'fin_loan_begin_dt', 'fin_loan_end_dt', 'stream_favorite_show',
       'stream_duration', 'create_date', 'update_date', 'source_cd',
       'source_priority', 'client_yo'],
      dtype='object')

In [229]:
GROUP_COLUMNS = ['client_fio_full', 'client_yo', 'client_cityzen', 'addr_region', 'addr_country', 'addr_city',
       'fin_rating', 'fin_loan_percent', 'stream_favorite_show', 'source_cd']

# GROUP_COLUMNS = [
#     'client_id',
#     'client_first_name',
#     'client_middle_name',
#     'client_last_name',
#     'client_fio_full',
#     'client_bday',
#     'client_inn',
#     'client_snils',
#     'contact_email',
#     'contact_phone',
# ]

In [230]:
def find_duplicates(df):
    # Используем LabelEncoder для текстовых колонок
    label_encoders = {}

    # Обработка колонок
    for col in GROUP_COLUMNS:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Создаем массив числовых фич для кластеризации
    features = df[GROUP_COLUMNS].values

    # Используем DBSCAN для поиска потенциальных дубликатов
    clustering = DBSCAN(eps=0.5, min_samples=2, metric='euclidean').fit(features)
    df['cluster'] = clustering.labels_

    # Обратное преобразование закодированных значений
    for col in GROUP_COLUMNS:
        le = label_encoders[col]
        df[col] = le.inverse_transform(df[col])

    return df

In [231]:
# Применение метода кластеризации
data = find_duplicates(data)

In [244]:
data = data[data['cluster'] != -1]
data[data['cluster'] == 4353]

,client_id,client_first_name,client_middle_name,client_last_name,client_fio_full,client_bday,client_bplace,client_cityzen,client_resident_cd,client_gender,...,fin_loan_begin_dt,fin_loan_end_dt,stream_favorite_show,stream_duration,create_date,update_date,source_cd,source_priority,client_yo,cluster
105324,851764452,гульсагида,камильжановна,гопкало,гопкало гульсагида камильжановна,1947-10-04,гор очамчира респ абхазия,rus,д,ж,...,,,,NaN,2021-11-29 17:31:06.634,2021-11-29 17:31:06.634,bank2,5,77.0,4353
198302,932455937,гульсагида,камильжановна,гопкало,гопкало гульсагида камильжановна,1947-10-04,гор очамчира респ абхазия,rus,д,ж,...,,,,NaN,2024-05-03 23:18:23.050,2024-05-03 23:18:23.050,bank2,5,77.0,4353


In [245]:
data

,client_id,client_first_name,client_middle_name,client_last_name,client_fio_full,client_bday,client_bplace,client_cityzen,client_resident_cd,client_gender,...,fin_loan_begin_dt,fin_loan_end_dt,stream_favorite_show,stream_duration,create_date,update_date,source_cd,source_priority,client_yo,cluster
79,367391504,милада,мязидовна,москинова,москинова милада мязидовна,1994-07-02,,,,ж,...,,,money heist,733.0,2022-01-03 23:26:47.918,2022-01-03 23:26:47.918,stream,0,30.0,0
92,78656073,ырсканым,жимбаевна,передирина,передирина ырсканым жимбаевна,1967-09-24,,,,ж,...,,,паразиты,1461.0,2023-11-20 19:51:47.452,2023-11-20 19:51:47.452,stream,0,57.0,1
121,336123512,джамалатин,варисович,фетхуллин,фетхуллин джамалатин варисович,1947-05-20,,,,м,...,,,lost,705.0,2021-03-31 00:00:39.112,2021-03-31 00:00:39.112,stream,0,77.0,2
137,807727318,кармэн,кизинбаевна,висарева,висарева кармэн кизинбаевна,1952-12-27,с.армизонское армизонского района тюменской об...,rus,д,ж,...,,,,NaN,2022-01-01 06:22:05.469,2022-01-01 06:22:05.469,bank2,5,71.0,3
164,329176390,,,,,1947-12-27,,,,м,...,,,,NaN,2023-10-24 19:09:53.263,2023-10-24 19:09:53.263,fitness,1,76.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005402,205354580,абдразак,феритович,хаврич,хаврич абдразак феритович,1931-02-01,,,,м,...,,,joker,1126.0,2020-12-18 05:49:19.646,2020-12-18 05:49:19.646,stream,0,93.0,447
1005418,777308245,хаваж,захиддин оглы,купри,купри хаваж захиддин оглы,1996-01-18,,,,м,...,,,1917,843.0,2022-07-17 07:29:23.304,2022-07-17 07:29:23.304,stream,0,28.0,2376
1005430,539314837,оня,гавунсовна,басс,басс оня гавунсовна,1965-08-16,,,,ж,...,,,список шиндлера,981.0,2021-05-19 16:35:38.414,2021-05-19 16:35:38.414,stream,0,59.0,1603
1005457,889029443,данияржан,ильфарович,тювелев,тювелев данияржан ильфарович,1954-04-24,,,,м,...,,,,1329.0,2020-05-25 09:17:54.217,2020-05-25 09:17:54.217,stream,0,70.0,2352


In [233]:
data['cluster'].unique()

array([     0,      1,      2, ..., 112790, 112791, 112792])

In [240]:
# Функция для выбора "золотой записи"
def select_golden_record(group: pd.DataFrame):
    # Пример: используем максимальную дату обновления и полноту полей как критерии
    group['completeness'] = group.notna().sum(axis=1)

    records_count = 3
    records = group.nlargest(records_count, ['source_priority', 'update_date', 'completeness'])

    golden_record = records.iloc[0]

    def is_valid_bday(bday: str):
        r = relativedelta(pd.to_datetime('today'), bday).years
        if r > 100 or r < 0:
            return False
        return True

    columns = records.columns
    for i, column in enumerate(columns):
        column_records = records[column].dropna()

        if column == 'client_bday':
            column_records = column_records[column_records.apply(is_valid_bday)]

        modes = column_records.mode()
        if len(modes) == 0:
            continue
        
        golden_record.iloc[i] = modes[0]

    return golden_record

In [241]:
# Группировка по кластерам и выбор золотой записи
golden_records = data.groupby('cluster', group_keys=False).apply(select_golden_record).reset_index(drop=True)

golden_records

/var/folders/fn/2pv4057d085149m8wfm5cnyw0000gn/T/ipykernel_38135/2167539832.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  golden_records = data.groupby('cluster', group_keys=False).apply(select_golden_record).reset_index(drop=True)


,client_id,client_first_name,client_middle_name,client_last_name,client_fio_full,client_bday,client_bplace,client_cityzen,client_resident_cd,client_gender,...,fin_loan_end_dt,stream_favorite_show,stream_duration,create_date,update_date,source_cd,source_priority,client_yo,cluster,completeness
0,367391504,милада,мязидовна,москинова,москинова милада мязидовна,1994-07-02,,,,ж,...,,money heist,733.0,2022-01-03 23:26:47.918,2022-01-03 23:26:47.918,stream,0,30.0,0,40
1,78656073,ырсканым,жимбаевна,передирина,передирина ырсканым жимбаевна,1967-09-24,,,,ж,...,,паразиты,147.0,2023-11-20 19:51:47.452,2023-11-20 19:51:47.452,stream,0,57.0,1,40
2,336123512,джамалатин,варисович,фетхуллин,фетхуллин джамалатин варисович,1947-05-20,,,,м,...,,lost,705.0,2021-03-31 00:00:39.112,2021-03-31 00:00:39.112,stream,0,77.0,2,40
3,654272368,кармэн,кизинбаевна,висарева,висарева кармэн кизинбаевна,1952-12-27,с.армизонское армизонского района тюменской об...,rus,д,ж,...,,,NaN,2022-01-01 06:22:05.469,2022-01-01 06:22:05.469,bank2,5,71.0,3,44
4,472156327,,,,,1948-03-12,,,,м,...,,,NaN,2024-07-15 10:16:13.824,2024-07-15 10:16:13.824,fitness,1,76.0,4,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112788,51404250,анес,исматилаевич,белушев,белушев анес исматилаевич,NaT,село колоденка,rus,д,м,...,,,NaN,2017-02-16 07:04:36.931,2017-02-16 07:04:36.931,bank2,5,62.0,112788,44
112789,54772254,мамлакат,загидуллаевна,добрышина,добрышина мамлакат загидуллаевна,1992-03-24,с. тея северо-енисейского р-на красноярского края,rus,д,ж,...,,,NaN,2017-09-12 10:04:09.009,2017-09-12 10:04:09.009,bank2,5,32.0,112789,45
112790,628512393,любчо,хамелюевич,бауло,бауло любчо хамелюевич,1967-01-28,г златоуст-36 челябинская обл,rus,д,м,...,,,NaN,2020-05-03 16:44:59.687,2020-05-03 16:44:59.687,bank2,5,57.0,112790,45
112791,422152367,гюнтер,зиядович,понамарчук,понамарчук гюнтер зиядович,1959-04-08,ст.сырт переволоукий,rus,д,м,...,,,NaN,2021-04-24 22:16:54.414,2021-04-24 22:16:54.414,bank2,5,65.0,112791,45


In [236]:
golden_records =  golden_records.drop(columns=['cluster', 'completeness', 'source_priority', 'client_yo'])

In [237]:
# Сохранение золотых записей в новый файл
golden_records.to_csv('golden_records_geg.csv', index=False)